Con questo codice si trasferisce il da file pdf a csv il file di domande di nostro interesse

In [ ]:
from PyPDF2 import PdfReader

pdf_path = "Area_Pre_Clinica_2016.pdf"
reader = PdfReader(pdf_path)

full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"


In [ ]:
import re

question_pattern = re.compile(
    r"(?P<id>\d{1,4})\.\s+(?P<question>.+?)\s+A(?P<correct_a>\*)?\s(?P<A>.+?)\s+"
    r"B(?P<correct_b>\*)?\s(?P<B>.+?)\s+"
    r"C(?P<correct_c>\*)?\s(?P<C>.+?)\s+"
    r"D(?P<correct_d>\*)?\s(?P<D>.+?)\s+"
    r"E(?P<correct_e>\*)?\s(?P<E>.+?)(?=\n\d+\.|\Z)",
    re.DOTALL
)


In [ ]:
def macro_category_from_id(qid: int) -> str:
    if 1 <= qid <= 395:
        return "farmacologia"
    elif 396 <= qid <= 1564:
        return "fisiologia"
    elif 1565 <= qid <= 1907:
        return "morfologia"
    elif 1908 <= qid <= 2822:
        return "patologia"
    elif 2823 <= qid <= 3344:
        return "prevenzione e deontologia medica"
    else:
        return ""


In [ ]:
questions = []
for match in question_pattern.finditer(full_text):
    qid = int(match.group("id"))
    correct = ""
    for letter in "abcde":
        if match.group(f"correct_{letter}"):
            correct = letter.upper()
            break

    questions.append({
        "id": qid,
        "question": match.group("question").strip().replace("\n", " "),
        "A": match.group("A").strip().replace("\n", " "),
        "B": match.group("B").strip().replace("\n", " "),
        "C": match.group("C").strip().replace("\n", " "),
        "D": match.group("D").strip().replace("\n", " "),
        "E": match.group("E").strip().replace("\n", " "),
        "answer": correct,
        "category": "Area Preclinica 2016",
        "macro category": macro_category_from_id(qid)
    })


In [ ]:
import csv

with open("area_preclinica_2016_domande.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=questions[0].keys())
    writer.writeheader()
    writer.writerows(questions)


In [ ]:
# Confrontiamo automaticamente le prime 20 domande tra PDF (testo estratto) e CSV

# Estrarre le prime 20 domande dal CSV
df_sample = df.head(20).copy()

# Estrarre lo stesso testo dal PDF già precedentemente letto
# Spezziamo le domande dal testo intero PDF (già caricato in `full_text`)
pdf_questions_raw = question_pattern.findall(full_text)

# Mappiamo l'output del PDF in un dizionario per confronto
pdf_data_map = {}
for q in pdf_questions_raw:
    qid = int(q[0])
    pdf_data_map[qid] = {
        "question": q[1].strip().replace("\n", " "),
        "A": q[3].strip().replace("\n", " "),
        "B": q[5].strip().replace("\n", " "),
        "C": q[7].strip().replace("\n", " "),
        "D": q[9].strip().replace("\n", " "),
        "E": q[11].strip().replace("\n", " "),
    }

# Confronta le prime 20 domande
differenze = []
for _, row in df_sample.iterrows():
    qid = row["id"]
    pdf_entry = pdf_data_map.get(qid)
    if not pdf_entry:
        continue

    diffs = {}
    for key in ["question", "A", "B", "C", "D", "E"]:
        csv_text = str(row[key]).strip()
        pdf_text = str(pdf_entry[key]).strip()
        if csv_text != pdf_text:
            diffs[key] = {"csv": csv_text, "pdf": pdf_text}
    
    if diffs:
        differenze.append({"id": qid, "diffs": diffs})

# Mostriamo le prime 3 differenze trovate
differenze[:3]
